#Right now uses mean of WnvPresent.
eventually, this should maybe rather use a counter? (like 0 or 1)

In [7]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, preprocessing, metrics
df_train = pd.read_csv('../Data_with_NumMosquitos/new_train_priors.csv')
#df_train_nomosq = pd.read_csv('../train_filled_new.csv')
df_test = pd.read_csv('../Data_with_NumMosquitos/new_test_priors.csv')
#df_test_nomosq = pd.read_csv('../test_filled_new.csv')
train_support = pd.read_csv('../train.csv')
test_support = pd.read_csv('../test.csv')

In [14]:
drop_species= True

In [15]:
df_train['Date'] = train_support.Date
df_train['Trap'] = train_support.Trap


In [16]:
tmp = df_train.drop(['WnvPresent','Id','Unnamed: 0'],axis=1)

In [17]:
len(tmp)

10506

In [23]:
train_support.groupby('Species').count()

,Date,Address,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
Species,,,,,,,,,,,
CULEX ERRATICUS,1,1,1,1,1,1,1,1,1,1,1
CULEX PIPIENS,2699,2699,2699,2699,2699,2699,2699,2699,2699,2699,2699
CULEX PIPIENS/RESTUANS,4752,4752,4752,4752,4752,4752,4752,4752,4752,4752,4752
CULEX RESTUANS,2740,2740,2740,2740,2740,2740,2740,2740,2740,2740,2740
CULEX SALINARIUS,86,86,86,86,86,86,86,86,86,86,86
CULEX TARSALIS,6,6,6,6,6,6,6,6,6,6,6
CULEX TERRITANS,222,222,222,222,222,222,222,222,222,222,222


In [24]:
test_support.groupby('Species').count()

,Id,Date,Address,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
Species,,,,,,,,,,
CULEX ERRATICUS,14345,14345,14345,14345,14345,14345,14345,14345,14345,14345
CULEX PIPIENS,14521,14521,14521,14521,14521,14521,14521,14521,14521,14521
CULEX PIPIENS/RESTUANS,15359,15359,15359,15359,15359,15359,15359,15359,15359,15359
CULEX RESTUANS,14670,14670,14670,14670,14670,14670,14670,14670,14670,14670
CULEX SALINARIUS,14355,14355,14355,14355,14355,14355,14355,14355,14355,14355
CULEX TARSALIS,14347,14347,14347,14347,14347,14347,14347,14347,14347,14347
CULEX TERRITANS,14351,14351,14351,14351,14351,14351,14351,14351,14351,14351
UNSPECIFIED CULEX,14345,14345,14345,14345,14345,14345,14345,14345,14345,14345


In [18]:

non_vectors = ['CULEX ERRATICUS','CULEX SALINARIUS','CULEX TARSALIS', 'CULEX TERRITANS']
if drop_species == True:
    for species in non_vectors:
        tmp = tmp[train_support.Species != species]
    
len(tmp)

/home/thomas/anaconda3/lib/python3.4/site-packages/pandas/core/frame.py:1819: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


10191

In [19]:
tmp2 = tmp.drop_duplicates()

In [20]:
len(tmp2)

8642

In [7]:
groups = ['Date','Trap','Species']

In [8]:
tmp = df_train.join(df_train.groupby(groups)['WnvPresent'].sum(), on= groups,rsuffix='_DateTrapSpecies')
tmp['WnvPresent_DateTrapSpecies'][tmp.WnvPresent_DateTrapSpecies > 0] = 1


/home/thomas/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [9]:
tmp.WnvPresent_DateTrapSpecies.min(),tmp.WnvPresent_DateTrapSpecies.max(),tmp.WnvPresent_DateTrapSpecies.median()

(0, 1, 0.0)

In [10]:
len(tmp)

10506

In [11]:
tmp2 = tmp.drop(['Trap','Id','Unnamed: 0','WnvPresent'],axis=1)

In [12]:
#use_for_duplicates = [var for var in tmp2.columns if var not in ['WnvPresent']]

In [13]:
tmp2 = tmp2.drop_duplicates()

In [14]:
tmp2 = tmp2.drop(['Date'],axis=1)

In [15]:
tmp2.shape

(8521, 97)

In [16]:
tmp2.to_csv('unique_train.csv',index_label='Id')